In [ ]:
%cd /kaggle/working

# in order to use mmdection ,we need to uninstall pytorch-1.9 and install pytorch-1.7

In [ ]:
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torch-1.7.0+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchvision-0.8.1+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchaudio-0.7.0-cp37-cp37m-linux_x86_64.whl' --no-deps

# we need to move mmdetection to kaggle working path.

In [ ]:
!cp -r ../input/mmdetectionv2140/* /kaggle/working/
!cp -r ../input/mmdetection-new/* /kaggle/working/
#your model
!cp -r ../input/handudu/mask_rcnn_cbv2_swin_tiny_patch4_window7_mstrain_480-800_adamw_3x_coco.py /kaggle/working/
!cp -r ../input/timtim/timm-0.4.12-py3-none-any.whl /kaggle/working/
!pip install '/kaggle/working/timm-0.4.12-py3-none-any.whl' --no-deps
!pip install '/kaggle/working/addict-2.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/working/yapf-0.31.0-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/working/terminal-0.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/working/terminaltables-3.1.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/working/mmcv_full-1_3_8-cu110-torch1_7_0/mmcv_full-1.3.8-cp37-cp37m-manylinux1_x86_64.whl' --no-deps
!pip install '/kaggle/working/pycocotools-2.0.2/pycocotools-2.0.2' --no-deps
!pip install '/kaggle/working/mmpycocotools-12.0.3/mmpycocotools-12.0.3' --no-deps
!mv /kaggle/working/CBNetV2-main  /kaggle/working/mmdetection
%cd /kaggle/working/mmdetection
!pip install -e .

# importing some tools are basic and important

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import sklearn
import torchvision
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import cupy as cp
import gc
import pandas as pd
import os
import matplotlib.pyplot as plt
import PIL
import json
from PIL import Image, ImageEnhance
import albumentations as A
import mmdet
import mmcv
from albumentations.pytorch import ToTensorV2
import seaborn as sns
import glob
from pathlib import Path
import pycocotools
from pycocotools import mask
import numpy.random
import random
from glob import glob
from tqdm.notebook import tqdm
import cv2
import re
import shutil
from mmdet.apis import inference_detector, init_detector, show_result_pyplot, set_random_seed

In [ ]:
%cd /kaggle/working

# add post function,such as rle trick in cell competition

In [ ]:

def rle_encoding(x):
    dots = np.where(x.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return ' '.join(map(str, run_lengths))
def get_mask_from_result(result):
    d = {True : 1, False : 0}
    u,inv = np.unique(result,return_inverse = True)
    mk = cp.array([d[x] for x in u])[inv].reshape(result.shape)
#     print(mk.shape)
    return mk
def does_overlap(mask, other_masks):
    for other_mask in other_masks:
        if np.sum(np.logical_and(mask, other_mask)) > 0:
            #import pdb; pdb.set_trace()
            #print("Found overlapping masks!")
            return True
    return False
def remove_overlapping_pixels(mask, other_masks):
    for other_mask in other_masks:
        if np.sum(np.logical_and(mask, other_mask)) > 0:
            print("Overlap detected")
            mask[np.logical_and(mask, other_mask)] = 0
    return mask

def one_hot(y, num_classes, dtype=cp.uint8): # GPU
    y = cp.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = cp.max(y) + 1
    n = y.shape[0]
    categorical = cp.zeros((n, num_classes), dtype=dtype)
    categorical[cp.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = cp.reshape(categorical, output_shape)
    return categorical

In [ ]:
def rle_encoding(x):
    dots = np.where(x.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return ' '.join(map(str, run_lengths))
def get_mask_from_result(result):
    d = {True : 1, False : 0}
    u,inv = np.unique(result,return_inverse = True)
    mk = cp.array([d[x] for x in u])[inv].reshape(result.shape)
#     print(mk.shape)
    return mk
def does_overlap(mask, other_masks):
    for other_mask in other_masks:
        if np.sum(np.logical_and(mask, other_mask)) > 0:
            #import pdb; pdb.set_trace()
            #print("Found overlapping masks!")
            return True
    return False
def remove_overlapping_pixels(mask, other_masks):
    for other_mask in other_masks:
        if np.sum(np.logical_and(mask, other_mask)) > 0:
            print("Overlap detected")
            mask[np.logical_and(mask, other_mask)] = 0
    return mask

In [ ]:
config_file = f'mask_rcnn_cbv2_swin_tiny_patch4_window7_mstrain_480-800_adamw_3x_coco.py'
checkpoint_file = '../input/final-model-w/0317.pth' 
model = init_detector(config_file, checkpoint_file, device='cuda:0')

In [ ]:
masks = []
files = []

MIN_PIXELS = {0: 75, 1: 75, 2: 150}
confidence_thresholds = {0: 0.25, 1: 0.65, 2: 0.35}
for imgs in tqdm(glob('../input/sartorius-cell-instance-segmentation/test/*')):
    result = inference_detector(model, imgs)
    
    pred_class_ls = [len(result[0][0]), len(result[0][1]), len(result[0][2])]
    pred_class = pred_class_ls.index(max(len(result[0][0]), len(result[0][1]), len(result[0][2])))
    msk = []
    for i, classe in enumerate(result[0]):
        if classe.shape != (0, 5):
            bbs = classe
            sgs = result[1][i]
            for bb, sg in zip(bbs,sgs):
                box = bb[:4]
                cnf = bb[4]
                if cnf >= confidence_thresholds[pred_class]:
                    mask = get_mask_from_result(sg)
                    mask = remove_overlapping_pixels(mask, msk)
                    if mask.sum() >= MIN_PIXELS[pred_class]:
                        msk.append(mask)
    for mk in msk:
            rle_mask = rle_encoding(mk)
            masks.append(rle_mask)
            files.append(str(imgs.split('/')[4].split('.')[0]))

In [ ]:
files = pd.Series(files, name='id')
preds = pd.Series(masks, name='predicted')
submission_df = pd.concat([files, preds], axis=1)
submission_df.to_csv('submission.csv', index=False)
submission_df.head()

In [ ]:
%cd /kaggle/working
!rm -rf /kaggle/working/mmcv_full-1_3_8-cu110-torch1_7_0
!rm -rf /kaggle/working/terminaltables-3.1.0-py3-none-any.whl
!rm -rf /kaggle/working/mmdetection
!rm -rf /kaggle/working/yapf-0.31.0-py2.py3-none-any.whl
!rm -rf /kaggle/working/mmdetection-2.14.0
!rm -rf /kaggle/working/yapf-0.31.0-py2.py3-none-any.whl
!rm -rf /kaggle/working/mmdetection-2.14.0
!rm -rf /kaggle/working/mmcv-1.3.8




In [ ]:
!rm -rf /kaggle/working/mmpycocotools-12.0.3
!rm -rf /kaggle/working/addict-2.4.0-py3-none-any.whl
!rm -rf /kaggle/working/pycocotools-2.0.2
!rm -rf /kaggle/working/mmcv-1.3.8
!rm -rf /kaggle/working/terminal-0.4.0-py3-none-any.whl
!rm -rf /kaggle/working/mmpycocotools-12.0.3
!rm -rf /kaggle/working/addict-2.4.0-py3-none-any.whl
!rm -rf /kaggle/working/pycocotools-2.0.2

In [ ]:

!rm -rf /kaggle/working/detectors_htc_r50_1x_coco.py
!rm -rf /kaggle/working/mask_rcnn_cbv2_swin_tiny_patch4_window7_mstrain_480-800_adamw_3x_coco.py
!rm -rf /kaggle/working/timm-0.4.12-py3-none-any.whl